In [1]:
from __future__ import print_function
import keras
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
import os

Using TensorFlow backend.


In [2]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:1 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:2 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:3 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:05:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:1 -> device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:06:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:2 -> device: 2, name: GeForce GTX 1080 Ti, pci bus id: 0000:09:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:3 -> device: 3, name: GeForce GTX 1080 Ti, pci bus id: 0000:0a:00.0, compute capability: 6.1



In [3]:
batch_size = 256
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20

In [4]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
x_test[0].dtype

dtype('uint8')

In [6]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(4, 4),
                 strides=1,
                 activation='relu',
                 input_shape=x_train.shape[1:]))
model.add(Conv2D(64, (4, 4), strides=2, activation='relu'))
model.add(Conv2D(128, (4, 4), strides=1, activation='relu'))
model.add(Conv2D(128, (4, 4), strides=1, activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

W0417 18:41:41.287202 140566470350656 deprecation.py:506] From /home/jiameng/packages/Bernsp/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
# initiate Adam optimizer
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [9]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [10]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=int(np.ceil(x_train.shape[0]/batch_size)),
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    workers=4)

Using real-time data augmentation.


W0417 18:41:43.639535 140566470350656 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/100
196/196 [==============================] - 23s 120ms/step - loss: 1.9775 - accuracy: 0.2431 - val_loss: 1.6322 - val_accuracy: 0.3763
Epoch 2/100
196/196 [==============================] - 21s 105ms/step - loss: 1.5736 - accuracy: 0.4164 - val_loss: 1.4175 - val_accuracy: 0.4809
Epoch 3/100
196/196 [==============================] - 21s 107ms/step - loss: 1.3892 - accuracy: 0.4880 - val_loss: 1.2811 - val_accuracy: 0.5355
Epoch 4/100
196/196 [==============================] - 21s 107ms/step - loss: 1.2695 - accuracy: 0.5381 - val_loss: 1.1590 - val_accuracy: 0.5862
Epoch 5/100
196/196 [==============================] - 21s 106ms/step - loss: 1.1643 - accuracy: 0.5797 - val_loss: 1.0476 - val_accuracy: 0.6309
Epoch 6/100
196/196 [==============================] - 21s 107ms/step - loss: 1.0794 - accuracy: 0.6125 - val_loss: 1.0515 - val_accuracy: 0.6247
Epoch 7/100
196/196 [==============================] - 21s 107ms/step - loss: 1.0147 - accuracy: 0.6384 - val_loss: 0.9394 -

Epoch 57/100
196/196 [==============================] - 21s 105ms/step - loss: 0.4234 - accuracy: 0.8532 - val_loss: 0.6139 - val_accuracy: 0.8018
Epoch 58/100
196/196 [==============================] - 20s 104ms/step - loss: 0.4238 - accuracy: 0.8520 - val_loss: 0.6074 - val_accuracy: 0.8089
Epoch 59/100
196/196 [==============================] - 20s 105ms/step - loss: 0.4220 - accuracy: 0.8525 - val_loss: 0.6287 - val_accuracy: 0.7975
Epoch 60/100
196/196 [==============================] - 21s 106ms/step - loss: 0.4248 - accuracy: 0.8533 - val_loss: 0.6114 - val_accuracy: 0.8079
Epoch 61/100
196/196 [==============================] - 22s 111ms/step - loss: 0.4147 - accuracy: 0.8570 - val_loss: 0.6143 - val_accuracy: 0.8032
Epoch 62/100
196/196 [==============================] - 22s 110ms/step - loss: 0.4141 - accuracy: 0.8580 - val_loss: 0.6017 - val_accuracy: 0.8052
Epoch 63/100
196/196 [==============================] - 21s 105ms/step - loss: 0.4093 - accuracy: 0.8584 - val_loss: 0

In [11]:
# serialize model to JSON
model_json = model.to_json()
with open("/home/jiameng/packages/ReachNN/ReachNN-CNN/model/model_CIFAR_CNN_Large.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/home/jiameng/packages/ReachNN/ReachNN-CNN/model/model_CIFAR_CNN_Large.h5")
print("Saved model to disk")

Saved model to disk
